# 4.1 The Y-86 ISA

### 4.1.1 Programmer-Visible State

The Y86-64 program can modify some part of the processor state. This is referred to as the *programmer visible state* where the programmer is someone writing programs in assembly.    
The state for Y86 is similar to that for x86. There are 15 program registers. There are three single bit condition codes for storing information, CF, ZF, OF.   
The **memory** is conceptually a large array of bytes. The Y84 programs reference memory locations using **virtual adresses**. Hardware and OS software translate these into actual or **physical addresses**. The status code indicates the state of program execution and indicates whether an exception has occurred.

### 4.1.2 Y86 Instructions

x86 movq instructions are split into irmovq, rrmovq, mrmovq, and rmmovq, the source isis either i, r, or m. The destination is r or m/ There are four integer operation instructions, operating only on reigster data. They set three condition codes, ZF, SF, and OF.
  
This is for Y86   

There are four integer operation instructions addq, subq, addq, and xorq.   

There are seven jump instructions, six conditional moves, and pushq, popq, and call. THere is even the halt instruction.

# 4.2 Logic Design and HCL

HCL is hardware control language, the language that is used to describe the control logic of the different processor designs.

### 4.2.1 Logic Gates

Logic gates are the basic computing elements for digital circuits and generate an ouput equal to a boolean function of the bit values of the inputs.   
Logic gates are always active.

### 4.2.2 Combinational Circuits

By assembling a number of logic gates, we can construct computational blocks known as combinational circuits.  
Outputs of two or more logic gates cannot be connected together, and the circuits must be acyclic for a certain circuit to be a combinational circuit.    

### 4.2.3 HCL expressions and Circuits

Typically, circuits are deisnged to operate on data words. These are groups of signals that represent integers or other control patterns.

### 4.2.5 Memory and Clocking

Circuits do not store information, they react to signals at their inputs. To create sequential circuits, we must introduce devices that sotre information.    
They will be controlled by a single **clock**, which determines when new values are loaded into the devices

**Clocked Registers** (or registers) store individual words or bits. The clock signal controls the loading and value of the register.      
  
**Random access memories** (or memories) store multiple words using an address to select which word should be read or written. Examples include the virtual memory system or the register file, in which the registers serve as addresses.

The register file has two read ports, and one write port. This allows multiple read and write operations to take place.

# 4.3. Sequential Y86 implementations

We describe a sequential processor called SEQ below. On each cycle, SEQ performs all the steps required to process a complete instructions.

### 4.3.1 Organizing Processing into stages

Processing an instruction involves many operations, which are organized into the following stages below:    
  
**Fetch**: The fetch stage reads the bytes of an instruction from memory, using the PC as the memory address. From the instruction, it extracts the two 4-bit portions of the instruction byte, the first part being icode (the instruction code) and the second ifun (instruction function). It possibly fetches a register specifier byte, which could be registers rA or rB, or a 4 byte constant word valC. It computes the address of instruction as the value of the PC plus the length of the instructions.  
  
**Decode**: The decode stage reads two operands from the register file, with values valA or valB. Sometimes it reads %esp, but most times it reads the registers in rA and rB.  
  
**Execute**: In the execute stage, the ALU performs the operation specified by the instructions (according to ifun), computes the effective address of reference, or increment and decrement the stack pointer. The condition codes might be set. For jumps, condition codes and branch conditions may be tested to see whether the branch should be taken.   
  
**Memory**: The memory stage may write data to memory or write data to memory.   
  
**Write Back**: The stage writes up to two results to the register file   
  
**PC Update**: The PC is set to the address of next instruction

The below is for **OPq, rrmovq, irmovq**
In the fetch stage, valP is always computed as PC+2, since we do not require a word in the fetch stage.  
During the decode stage we read operands rA and rB. These two operands are spplied to the ALU in the execute stage with ifun, which describes the operand to be applied, and **valE becomes the result of the instruction**.  
Nothing happens in the memory stage since nothing is written to memory. However, in write-back, valE is written to register rB, and the PC is set to valP to complete execution. 

### 4.3.2 SEQ Hardware Structure

The computations required to implement all the Y86 instructions can be separated into sex stage: fetch, decode, execute, memory, write back and PC Update.  
The PC is usually stored in a register. Information then flows along wires. Processing is performed by hardware units associated with their respective stages.   
In SEQ, all processing by the hardware units occur within a single clock cycle.   
Th hardware units associated with the below stages are:   
**FETCH**: Using the program counter register as the address, the instruction memory reads the bytes of instruction, the PC incrementer computes valP, the resulting PC.   
**DECODE**: The register file has 2 read ports, which have 2 values valA and valB which are read simultaneously.   
**EXECUTE**: The execute stage uses the ALU for different purposes according the the instruction type, and changes the condition code registers.   
**MEMORY**: The data reads or writes words of memory when executing an instruction. The instruction and data memories access the same mmeory location but for different purposes.   
**WRITE BACK**: The register file has 2 write ports. Port E is used to write values computed by the ALU, while port M is used to read from data memory.   
**PC UPDATE** - The new value of the PC is either valP, the destination address, or valM, the return address

Tidbits:  
PC is the only clocked register in SEQ.  
Hardware units include the memories, the ALU, and so forth.  
Control logic blocks serve to select from signal sources and compute some functions.   

### 4.3.3 SEQ Timing

Our SEQ Processor consists of two memory devices, clocked registers (program counter) and random access memories (program file, data memories, and instruction memories).  
It also includes four hardware units that require explicit control over their sequencing, the PC, the condition code register, the data memory, and the register file. They are controlled by a clock signal that signals when to load new values into registers and write new values to the random access memories. The PC is loaded with a new instruction every clock cycle.   
The clocking of registers and memories is all that is required to cnotrol the sequencing of activities in our processors. 

The hardware achieves the same effect on the same set of instructions, because there is no reading back.  
**PRINCIPLE 1**: No Reading Back.  
**EXAMPLE**: If you had a pushq instruction, and we violated the principle by reading back in our implementation of pushq, we would do the following:   
We would decrement rsp first (push instruction decrements rsp), and then use that updated value of rsp as the address of the write operation. This would require reading the stack pointer from the register file to perform the memory operation.  
Rather than doing this, we generate the decremented stack pointer and save it as an intermediary value and then use it for memory write and read, which we can do concurrently in one cycle.

### 4.3.4 SEQ Stage Implementations

**Fetch Stage**: The fetch stage includes the instruction memory hardware unit, which reads 10 bytes from memory at a time, using the PC as the starting byte. The first byte is interpreted as the instruction byte read and then split into 2 4 bit quantities. The logic blocks known as icode and ifun then compute the instruction and function codes.   
  
Based on the value of icode, three signal codes can be computed, **instr_valid**, which detects an illegal instruction, **need_regids**, which indicates if this instruction includes a register specifier byte, and **need_valC** which indicates if this instruction includes a constant word.  
The remaining 9 bytes read from the instruction could encodethe combination of a register specifier byte or a word. These bytes are processed in a hardware unit labeled Align and separated into the register field and the constant word. Byte 1 is split into the register specifiers rA and rB if need_regids is 1. 